<a href="https://colab.research.google.com/github/Mohit352-bit/Machine-Learning/blob/main/Neural%20Network/Transfer_Learning_and_Dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os



In [2]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

#storing the path of weight file
weights_file_path = 'C:/Users/mohit/Downloads/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'


#creating a model with inceptionV3 with no weights
pre_trained_model = InceptionV3(input_shape=(300,300,3),
                               include_top= False,
                               weights = None)


#adding downloaded weights to the model created
pre_trained_model.load_weights(weights_file_path)

#freezing each layer of the model so that weights can't be changes further
for layers in pre_trained_model.layers:
    layers.trainable = False

In [3]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [4]:
# we choose 'mixed 7' as we want only the model till where it is connected to mixed7 where shape is (71,17,768)
last_layer = pre_trained_model.get_layer('mixed7')
last_lyr_output = last_layer.output
print("shape of last layer of pre trained model" ,last_layer.output_shape)
last_lyr_output

shape of last layer of pre trained model (None, 17, 17, 768)


<KerasTensor: shape=(None, 17, 17, 768) dtype=float32 (created by layer 'mixed7')>

# Tranfer Learning

In [14]:
mod = tf.keras.applications.InceptionV3(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

In [15]:
ip=mod.layers[0].input
op=mod.layers[-2].output


In [16]:
op

<KerasTensor: shape=(None, 2048) dtype=float32 (created by layer 'avg_pool')>

## Dropout
In dropout few nodes are droped from a layer which have nearly same weight as the other nodes so as to reduce the overfiting of the model.

In [17]:
#connecting the pre_trained_weight model with the DNN 

#second part show the connection of the last layer 
x = tf.keras.layers.Flatten()(op)
x = tf.keras.layers.Dense(512 , activation='relu')(x)

#using dropout
x = tf.keras.layers.Dropout(0.2)(x)
X = tf.keras.layers.Dense(1, activation="sigmoid")(x)


In [20]:
from tensorflow.keras import Model

model=tf.keras.Model(inputs =ip,outputs=x)

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 149, 149, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 149, 149, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 149, 149, 32) 0           batch_normalization_188[0][0]    
______________________________________________________________________________________________

In [6]:
model = tf.keras.Sequential([
  last_lyr_output,
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512 , activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1, activation="sigmoid")  
    
])

TypeError: The added layer must be an instance of class Layer. Found: KerasTensor(type_spec=TensorSpec(shape=(None, 17, 17, 768), dtype=tf.float32, name=None), name='mixed7/concat:0', description="created by layer 'mixed7'")

In [22]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [30]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
val_datagen = ImageDataGenerator(rescale=1./255)

train_datagen = train_datagen.flow_from_directory('C:/Users/mohit/Downloads/Compressed/cats_and_dogs_filtered/cats_and_dogs_filtered/train',
                                                    batch_size = 10,
                                                    class_mode = 'binary', 
                                                    target_size = (300, 300)) 
val_datagen = val_datagen.flow_from_directory( 'C:/Users/mohit/Downloads/Compressed/cats_and_dogs_filtered/cats_and_dogs_filtered/validation',
                                                          batch_size  = 10,
                                                          class_mode  = 'binary', 
                                                          target_size = (300, 300))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [31]:
history = model.fit(
            train_datagen,
            validation_data = val_datagen,
            steps_per_epoch = 10,
            epochs = 20, batch_size=10)

Epoch 1/20


ResourceExhaustedError:  OOM when allocating tensor with shape[10,96,17,17] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/conv2d_217/Conv2D (defined at <ipython-input-24-0c104b721880>:5) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_29311]

Function call stack:
train_function


In [27]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model
from tensorflow.keras.layers import Flatten, Dense , Dropout

# Flatten the output layer to 1 dimension
x = Flatten()(last_lyr_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
y = Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
z = Dropout(0.2)(y)                  
# Add a fial sigmoid layer for classification
w = Dense  (1, activation='sigmoid')(z)           

# Append the dense network to the base model
model = Model(pre_trained_model.input, w) 

ResourceExhaustedError: OOM when allocating tensor with shape[221952,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]